- **property list (_plist_)** - where every other element, starting with the first, is a symbol that describes what the next element in the list is.
```lisp
(list :a 1 :b 2 :c 3)
```

- Can define a **plist** with the same **list** function used for normal lists, it's the contents that make it a **plist**.

- **GETF** function takes a **plist** and a symbol and returns the value in the **plist** following the symbol.
```lisp
(getf (list :a 1 :b 2 :c 3) :b)  ; => 2
```

- Can use **PUSH** to push things into a list.
```lisp
(push 34 data)
```

> #### A bit about **format**
> ```lisp
(format t "~{~a:~10t~a~%~}~%" cd)
```
> - **t** is shorthand for **\*standard-output\***
> - Formatting directives start with **~**
> - The **~a** directive is the _aesthetic_ directive. Consumes a single argument and outputs it in human readable form.
> - The **~t** directive is for tabulating. **~10t** means emit enough spaces to move to the tenth column before processing the next directive. Doesn't consume arguments.
> - The **~{** implies that the next argument to be consumed must be a list. **format** loops over the list, consuming as many elements of the list as needed each time through the list. The **~}** ends the loop.
> - The **~%** does not consume arguments and tells **format** to emit a newline.

#### Reading input
```lisp
(defun prompt-read (prompt)
  (format *query-io* "~a: " prompt)
  (force-output *query-io*)
  (read-line *query-io*))
```
- **prompt-read** takes in a string prompt that it displays to the user in a human-readable format.
- The call to **force-output** is necessary in some Lisp implementations to ensure it doesn't wait for a newline before it prints the prompt.
- You can read a single line of text using **read-line**
- The variable **\*query-io\*** is a global variable that contains the input stream connected to the terminal.
- The return value of **prompt-read** will be the last form, the call to **read-line**, which returns the read string without the trailing newline.

#### Saving to a file
```lisp
(defun save-db (filename)
  (with-open-file (out filename
                        :direction :output
                        :if-exists :supersede)
    (with-standard-io-syntax
      (print *db* out))))
```
- **with-open-file** macro opens a file, binds the stream to a variable, executes a set of expressions and finally closes the file. Ensures file is closed even if something went wrong in the body of the function.
- List directly after **with-open-file** isn't a function call - part of the syntax defined by **with-open-file**.

    Contains:
        - name of the variable that wil hold the file stream to which you'll write to in the body of the **with-open-file**.
        - A value that must be a filename.
        - Options that control how the file is opened
            - :direction :output - specify you're opening the file for writing.
            - :if-exists :supersede - specify that you want to overwrite the file if it exists.
- **print**, unlike **format**, outputs Lisp objects in a form that can be read back in by the Lisp reader (think, **_repr_** in Python).
- **with-standard-io-syntax** ensures that certain variables that affect the behaviour of **print** are set to their standard values.

#### Reading from a file
```lisp
(defun load-db (filename)
  (with-open-file (in filename)
    (with-standard-io-syntax
      (setf *db* (read in)))))
```
- No need to specify **:direction** in **with-open-file** since the default is **:input**.
- Use the **read** function to read from the stream *in*.
- **with-standard-io-syntax** ensures **read** is using the same basic syntax **print** used when saving.
- **setf** is Common Lisp's main assignment operator. Sets the first argument to the result of evaluating the second argument.

#### Querying the database
- **remove-if-not** - Takes a predicate fn and a list and returns a new list containing only the elements of the original list that match the predicate. (think, **_Array.filter_** in JavaScript).
- **evenp** - Predicate function - true if arg is an even number
- **#'** - Shorthand for **function**. Means "Get me the function with this name".
```lisp
(remove-if-not #'evenp '(1 2 3 4 5 6 7 8 9 10))  ; ==> (2 4 6 8 10)
(remove-if-not (function evenp) '(1 2 3 4 5 6 7 8 9 10))  ; ==> (2 4 6 8 10)
```

#### Quoting
- Using **quote** yields expressions without evaluating them (passed as they are).
```lisp
(quote (1 2 3 4 this that them))  ; ==> (1 2 3 4 THIS THAT THEM)
```
- You can use the single quote (**'**) as a shorthand for **quote**.
```lisp
'(1 2 3 4 this that them)  ; ==> (1 2 3 4 THIS THAT THEM)
```
```lisp
(let ((field :rating) (value 10))
    (list 'equal (list 'getf 'cd field) value))
; ==> (EQUAL (GETF CD :RATING) 10)
```
```lisp
'(1 2 3 (+ 3 4) 5)  ; ==> (1 2 3 (+ 3 4) 5)
```

#### Backquote
- Allows you to **quote** a list, but selectively evaluate elements in that list. In the simplest form, it's identical to **quote**.
```lisp
`(1 2 3 (+ 3 4) 5) ; ==> (1 2 3 (+ 3 4) 5)
```
- Using the special comma marker '**`,`**' inside of the argument to backquote indicates that a value is not a constant and it gets evaluated. The result of that evaluation is used in the quoted list. 
```lisp
`(1 2 3 ,(+ 3 4) 5) ; ==> (1 2 3 7 5)
```
> any subexpression preceded by a comma gets evaluated.

```lisp
(let ((field :rating) (value 10))
    `(equal (getf cd ,field) ,value))
; ==> (EQUAL (GETF CD :RATING) 10)
```

- Using a variant of **`,`** namely **`,@`**, the value of the evaluated expression gets spliced into the enclosing list. So, basically, **`,@`** "splices".
```lisp
`(and ,(list 1 2 3))  ; ==> (AND (1 2 3))
`(and ,@(list 1 2 3)) ; ==> (AND 1 2 3)
```

- Can also use **`,@`** to splice in the middle of a list
```lisp
`(and ,@(list 1 2 3) 4) ; ==> (AND 1 2 3 4)
```